<a href="https://colab.research.google.com/github/adrien-chinour/ia-data/blob/master/07-convolutional-neural-network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN)

In [0]:
# import librairies

import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import RMSprop
import numpy as np

Using TensorFlow backend.


## Exercice 1 : Un premier réseau de neurones

Dans cet exercice, nous allons définir un réseau avec une couche cachée contenant 1 neurone et
une couche de sortie avec deux neurones. Le but est toujours de faire une classification binaire et
arriver à faire la différence entre le chiffre 5 et les autres chiffres.
1. Modifier le fichier pour implémenter ce réseau de neurones. Quelles fonctions d’activation utiliseriezvous pour la couche cachée ? Pour la couche de sortie ?
2. Entrainer le réseau avec juste 10 epchos, le tester et mesurer sa précision. Peut-on améliorer
encore la précision sans changer l’architecture du réseau ?

In [0]:
#load (first download if necessary) the MNIST dataset
# (the dataset is stored in your home direcoty in ~/.keras/datasets/mnist.npz
#  and will take  ~11MB)
# data is already split in train and test datasets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train : 60000 images of size 28x28, i.e., x_train.shape = (60000, 28, 28)
# y_train : 60000 labels (from 0 to 9)
# x_test  : 10000 images of size 28x28, i.e., x_test.shape = (10000, 28, 28)
# x_test  : 10000 labels
# all datasets are of type uint8


#To input our values in our network Dense layer, we need to flatten the datasets, i.e.,
# pass from (60000, 28, 28) to (60000, 784)
#flatten images
num_pixels = x_train.shape[1] * x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], num_pixels)
x_test = x_test.reshape(x_test.shape[0], num_pixels)

#Convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Normalize inputs from [0; 255] to [0; 1]
x_train = x_train / 255
x_test = x_test / 255


#We want to have a binary classification: digit 0 is classified 1 and 
#all the other digits are classified 0

y_new = np.zeros(y_train.shape)
y_new[np.where(y_train==5.0)[0]] = 1
y_train = y_new

y_new = np.zeros(y_test.shape)
y_new[np.where(y_test==5.0)[0]] = 1
y_test = y_new


num_classes = 2

In [0]:
# Création du réseau de neurone à deux couches (1-2)
def deep_neural_network_1():
  nn = Sequential()
  nn.add(Dense(1, input_dim=num_pixels, activation='relu', kernel_initializer='normal'))
  nn.add(Dense(2, activation='softmax', kernel_initializer='normal'))
  nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
  return nn

In [0]:
# Tri des résultats (y) en deux catgéories 5 et !5
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [0]:
# On entraine notre modèle
my_network = deep_neural_network_1()
my_network.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 1)                 785       
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 4         
Total params: 789
Trainable params: 789
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.3384 - acc: 0.9072 - val_loss: 0.2347 - val_acc: 0.9108
Epoch 2/10
60000/60000 [==============================] - 1s 16us/step - loss: 0.2119 - acc: 0.9097 - val_loss: 0.1858 - val_acc: 0.9108
Epoch 3/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.1795 - acc: 0.9097 - val_loss: 0.1638 - val_acc: 0.9108
Epoch 4/10
60000/60000 [==============================] 

In [0]:
# Score obtenu par notre modèle
score = my_network.evaluate(x_test, y_test)
print("Accuracy : %.2f%%" %(score[1]*100))

10000/10000 [==============================] - 0s 21us/step
Accuracy : 96.83%


## Exercice 2 : Un réseau pour reconnaitre tous les chiffres

À présent, nous allons adapter le réseau précédent pour reconnaire tous les 10 chiffres.
1. Expliquer, avec une figure, l’architecture du nouveau réseau.
2. Redéfinir, dans le programme, la nouvelle architecture.
3. Entrainer et tester le nouveau réseau. Refaire les mêmes experimentations que dans l’exercice 1. Changer d’algorithme d’optimisation

In [0]:
# Récupération de notre jeu de données
(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_pixels = x_train.shape[1] * x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], num_pixels)
x_test = x_test.reshape(x_test.shape[0], num_pixels)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [0]:
# Création du réseau de neurone à deux couches (1-10)
def deep_neural_network_2():
  nn = Sequential()
  nn.add(Dense(1, input_dim=num_pixels, activation='relu', kernel_initializer='normal'))
  nn.add(Dense(10, activation='softmax', kernel_initializer='normal'))
  nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
  return nn

In [0]:
# On entraine notre modèle
my_network = deep_neural_network_2()
my_network.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 2s 35us/step - loss: 2.2936 - acc: 0.1053 - val_loss: 2.2680 - val_acc: 0.1117
Epoch 2/10
60000/60000 [==============================] - 1s 22us/step - loss: 2.1843 - acc: 0.1465 - val_loss: 2.0918 - val_acc: 0.2176
Epoch 3/10
60000/60000 [==============================] - 1s 20us/step - loss: 2.0355 - acc: 0.2079 - val_loss: 1.9861 - val_acc: 0.2114
Epoch 4/10
60000/60000 [==============================] - 1s 22us/step - loss: 1.9608 - acc: 0.2203 - val_loss: 1.9316 - val_acc: 0.2200
Epoch 5/10
60000/60000 [==============================] - 1s 19us/step - loss: 1.9171 - acc: 0.2311 - val_loss: 1.8974 - val_acc: 0.2286
Epoch 6/10
60000/60000 [==============================] - 1s 19us/step - loss: 1.8877 - acc: 0.2396 - val_loss: 1.8734 - val_acc: 0.2408
Epoch 7/10
60000/60000

In [0]:
# Score obtenu par notre modèle
score = my_network.evaluate(x_test, y_test)
print("Accuracy : %.2f%%" %(score[1]*100))

10000/10000 [==============================] - 0s 26us/step
Accuracy : 26.63%


## Exercice 3 : Un réseau CNN pour reconnaitre tous les chiffres
Même exercice mais en utilisant une architecture en CNN.
1. Définir un réseau de neurones à convolution avec :
  - une couche de convolution
  - une couche de maxpooling
  - un réseau "fully connected" à plusieurs couches
2. Entrainer le modèle et le tester.
3. Choisir les bons hyper-paramètres pour atteindre une précision d’u moins 99%.

In [0]:
# Récupération du jeu de données
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [0]:
# Création de notre CNN
def cnn():
  nn = Sequential()
  nn.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
  nn.add(MaxPool2D(pool_size=(2,2)))
  nn.add(Flatten())
  nn.add(Dense(10, activation='softmax', kernel_initializer='normal'))
  nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return nn

In [13]:
# Entrainement de notre réseau
my_network = cnn()
my_network.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 26s 442us/step - loss: 0.2825 - acc: 0.9198 - val_loss: 0.1208 - val_acc: 0.9642
Epoch 2/10
60000/60000 [==============================] - 26s 430us/step - loss: 0.1034 - acc: 0.9704 - val_loss: 0.0758 - val_acc: 0.9757
Epoch 3/10
60000/60000 [==============================] - 26s 437us/step - loss: 0.0739 - acc: 0.9789 - val_loss: 0.0656 - val_acc: 0.9790
Epoch 4/10
60000/60000 [==============================] - 26s 436us/step - loss: 0.0601 - acc: 0.9822 - val_loss: 0.0639 - val_acc: 0.9781
Epoch 5/10
60000/60000 [==============================] - 26s 428us/step - loss: 0.0510 - acc: 0.9847 - val_loss: 0.0618 - val_acc: 0.9786
Epoch 6/10
60000/60000 [==============================] - 26s 430us/step - loss: 0.0455 - acc: 0.9865 - val_loss: 0.0544 - val_acc: 0.9805
Epoch 7/10
60000/60000 [==============================] - 26s 430us/step - loss: 0.0400 - acc: 0.9883 - val_loss: 0.

In [14]:
# Score obtenu par notre modèle
score = my_network.evaluate(x_test, y_test)
print("Accuracy : %.2f%%" %(score[1]*100))

10000/10000 [==============================] - 2s 161us/step
Accuracy : 98.29%
